In [ ]:
!pip install gradio openai pandas numpy scikit-learn


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = input("Enter your OpenAI API key: ").strip()

if os.environ["OPENAI_API_KEY"]:
    print("✅ API key set")
else:
    print("⚠️ API key is empty!")


In [ ]:
import os
import numpy as np
import pandas as pd
import gradio as gr
from openai import OpenAI

# -----------------------------
# OpenAI client
# -----------------------------
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY is not set. Please run the API key cell first.")

client = OpenAI(api_key=api_key)

# -----------------------------
# Load Product Dataset
# -----------------------------
products_data = [
    [101,"Nimbus X15 Laptop","Electronics","Laptop","TechWave",52000,4.4,"work,student","i5 processor, 8GB RAM, 512GB SSD"],
    [102,"AuraBook Lite","Electronics","Laptop","Skyline",43000,4.1,"student,budget","Lightweight laptop ideal for students"],
    [103,"PixelTune Earbuds","Electronics","Audio","SoundMax",2999,4.3,"wireless,music","Noise isolation, 24-hour battery"],
    [104,"BassPro Headphones","Electronics","Audio","SoundMax",1799,4.0,"gaming,music","Deep bass wired headphones"],
    [105,"FitTrack Pro Smartwatch","Wearables","Smartwatch","FitTrack",3999,4.2,"fitness,tracking","Heart-rate & step-tracker"],
    [106,"UrbanMove Sneakers","Fashion","Footwear","UrbanWalk",2499,4.1,"casual","Daily wear sneakers"],
    [107,"ComfortStride Running Shoes","Fashion","Footwear","SprintX",3199,4.5,"running","Extra cushioning, breathable"],
    [108,"Classic Cotton T-Shirt","Fashion","Apparel","WearWell",699,4.0,"casual","100% cotton t-shirt"],
    [109,"EcoSip Steel Bottle","Home & Kitchen","Bottle","EcoSip",699,4.6,"travel,office","Steel insulated bottle"],
    [110,"OfficePro Ergonomic Chair","Home & Office","Chair","SitRight",8499,4.3,"office","Lumbar support ergonomic chair"],
]

product_df = pd.DataFrame(
    products_data,
    columns=[
        "product_id","name","category","subcategory","brand",
        "price","rating","tags","description"
    ]
)

# -----------------------------
# Product Recommendation Logic
# -----------------------------
def recommend_products(category: str, budget: float | None, min_rating: float):
    df = product_df.copy()

    if category != "Any":
        df = df[df["category"] == category]

    if budget is not None and budget > 0:
        df = df[df["price"] <= budget]

    df = df[df["rating"] >= min_rating]

    # fallback if no match
    if df.empty:
        df = product_df.nlargest(3, "rating")
    else:
        df = df.sort_values(["rating", "price"], ascending=[False, True]).head(3)

    return df

def explain_recommendations(user_need: str, rec_df: pd.DataFrame) -> str:
    if rec_df.empty:
        return "No products found."

    products_text = ""
    for _, row in rec_df.iterrows():
        products_text += (
            f"- {row['name']} ({row['brand']}): ₹{row['price']}, "
            f"⭐ {row['rating']} – {row['description']}\n"
        )

    prompt = f"""
You are an AI product recommendation expert.

User need:
{user_need if user_need.strip() else "Not specified"}

Recommended products:
{products_text}

Explain in simple and friendly language why these products are good choices.
"""

    try:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
        )
        return resp.choices[0].message.content
    except Exception as e:
        return f"Error calling OpenAI API: {e}"

def gr_product(category, budget, min_rating, user_need):
    budget = float(budget) if budget is not None else None
    recs = recommend_products(category, budget, min_rating)
    explanation = explain_recommendations(user_need, recs)

    return recs[["name","brand","category","price","rating","description"]], explanation

# -----------------------------
# Build Gradio UI
# -----------------------------
categories = ["Any"] + sorted(product_df["category"].unique().tolist())

with gr.Blocks(theme="soft") as demo:
    gr.Markdown("# 🛒 Product Recommendation Agent")

    gr.Markdown("Get product suggestions based on category, budget, and rating.")

    with gr.Row():
        prod_cat = gr.Dropdown(choices=categories, value="Any", label="Category")
        prod_budget = gr.Number(value=5000, label="Max Budget (₹)")
        prod_rating = gr.Slider(minimum=1.0, maximum=5.0, value=4.0, step=0.1, label="Minimum Rating")

    user_need = gr.Textbox(
        label="User Need (optional)",
        lines=3,
        placeholder="Example: I need a lightweight laptop for online classes."
    )

    prod_table = gr.Dataframe(
        label="Top Recommendations",
        headers=["name","brand","category","price","rating","description"],
        row_count=3,
        col_count=6
    )

    prod_explanation = gr.Textbox(label="Why These Products?", lines=6)

    prod_button = gr.Button("Recommend")

    prod_button.click(
        fn=gr_product,
        inputs=[prod_cat, prod_budget, prod_rating, user_need],
        outputs=[prod_table, prod_explanation]
    )

demo.launch(share=True)
